In [293]:
import pandas as pd
import re
import string
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.util import ngrams
#from google.colab import drive
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import warnings
warnings.filterwarnings('ignore')

In [294]:
aug = True

In [295]:
train_data_file_m = '/data/parush/stance_mohammed/new_train.txt'
test_data_file_m = '/data/parush/stance_mohammed/new_test.txt'
TARGETS_m = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion', 'Donald Trump']


# train_data_file_s = '/data/parush/SomasundaranWiebe-politicalDebates/train.txt'
# test_data_file_s = '/data/parush/SomasundaranWiebe-politicalDebates/test.txt'
# TARGETS_s = ['god','healthcare','guns','gayRights','abortion', 'creation']


# train_data_file_q = '/data/parush/Data_MPCHI/train.txt'
# test_data_file_q = '/data/parush/Data_MPCHI/test.txt'
# TARGETS_q = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
#       'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
#       'Should women take HRT post-menopause?']
# # Loading Data
# df = pd.read_csv(train_data_file, sep='\t')
# df1 = pd.read_csv(test_data_file, sep='\t')



# #Use below lines only when training different classifiers for different targerts and testing on their corrosponding data.
# # t = ['god','healthcare','guns','gayRights','abortion', 'creation']
# # df = df[(df["Target"]==t[1]) ]
# # df1 = df1[(df1["Target"]==t[1]) ]

# print("The length of train data is {}".format(len(df)))
# print("The length of test data is {}".format(len(df1)))

# #dummy
# import numpy as np
# from sklearn.dummy import DummyClassifier
# X = y_train
# y = y_test

# dummy_clf = DummyClassifier(strategy="most_frequent")
# dummy_clf.fit(X,X)


# y_true, y_pred = y_test, dummy_clf.predict(y)
# print(classification_report(y_true, y_pred))


In [296]:
vectorizer = 'tfidf'   # set 'count' or 'tfidf'
analyzer = 'both'  # set 'word' or 'both' ( word and char)

In [297]:
if vectorizer == 'count':
    if analyzer == 'word':
        vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,1))
    else:
        vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,3))
        char_vectorizer = CountVectorizer(analyzer='char',ngram_range=(2,5))
else:
    if analyzer == 'word':
        vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,1))
    else:
        vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,3))
        char_vectorizer = TfidfVectorizer(analyzer='char',ngram_range=(2,5))
        
        
        
        

In [304]:
#List of FAVOR Tweets
def get_training_data_and_labels(file,target):
    train_corpus = []
    train_labels = []
    classes = {'FAVOR': 0, 'AGAINST': 1, 'NONE': 2}
    with open(file) as f:
        for line in f:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
            if line[0].strip() != 'ID' and target in line[1].strip():
            #if line[0].strip() != 'ID':
                
                tweet = line[2]
                stance = line[3]
                
                train_corpus.append(tweet)
                train_labels.append(classes[stance])
    c_l = len(train_corpus)
    print("Current length",c_l)
    if aug:
        with open('rich_data/dt_ath_hc/Donald Trump_Atheism_Hillary Clinton0.9.txt','r') as newfile:
            for line in newfile:
                line = line.replace('#SemST', '').strip()
                line = line.split('\t')
                tweet = line[0]
                stance = line[1]
                #tweet = process_tweet(tweet)
                train_corpus.append(tweet)
                train_labels.append(classes[stance.strip()])
    print("Added more labels", len(train_corpus) - c_l )
    
    if analyzer == 'word':
        ngram_vectorized_data = vectorizer.fit_transform(train_corpus)
        
        return ngram_vectorized_data, train_labels
    else:
        ngram_vectorized_data = vectorizer.fit_transform(train_corpus)
        char_vectorized_data = char_vectorizer.fit_transform(train_corpus)
        l = np.hstack((ngram_vectorized_data.toarray(), char_vectorized_data.toarray()))
        train_vectorized_data = sparse.csr_matrix(l)
        
        return train_vectorized_data, train_labels 


In [305]:
#preparing test_data
def get_test_data_and_labels(file,target):
    test_corpus = []
    test_labels = []
    classes = {'FAVOR': 0, 'AGAINST': 1, 'NONE': 2}
    with open(file) as f:
        for line in f:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
        #if line[0].strip() != 'ID' and line[3].strip() == 'FAVOR' and line[1] == t:
            if line[0].strip() != 'ID' and target in line[1].strip():
                tweet = line[2]
                stance = line[3]
                #print(line[0])
                test_corpus.append(tweet)
                test_labels.append(classes[stance])
    if analyzer == 'word':
        test_ngram_vectorized_data = vectorizer.transform(test_corpus)
        
        return test_ngram_vectorized_data, test_labels
    else:
        test_ngram_vectorized_data = vectorizer.transform(test_corpus)
        test_char_vectorized_data = char_vectorizer.transform(test_corpus)
        l2 = np.hstack((test_ngram_vectorized_data.toarray(), test_char_vectorized_data.toarray()))
        test_vectorized_data = sparse.csr_matrix(l2)
        
        
        return test_vectorized_data,test_labels
    
    
    
    

In [306]:
X_train, y_train =  get_training_data_and_labels(train_data_file_m, TARGETS_m[4])
X_test, y_test = get_test_data_and_labels(test_data_file_m,TARGETS_m[3])


Current length 653
Added more labels 248


In [307]:
unique,count = np.unique(y_train,return_counts=True)
dict(zip(unique, count))
unique,count = np.unique(y_test,return_counts=True)
dict(zip(unique, count))

{0: 45, 1: 172, 2: 78}

In [308]:
from sklearn.model_selection import StratifiedKFold

In [309]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
scores = ['precision', 'recall']


for score in scores:
    
    print("# Tuning hyper-parameters for %s" % score)
    print()
    cv = StratifiedKFold(n_splits=2, shuffle=True, random_state = 2 )
    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score, cv = cv
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred, digits = 4, labels = [0,1]))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 1, 'kernel': 'linear'}

Grid scores on development set:

0.192 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.192 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.192 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.192 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.192 (+/-0.000) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.192 (+/-0.000) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.686 (+/-0.020) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.192 (+/-0.000) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.689 (+/-0.014) for {'C': 1, 'kernel': 'linear'}
0.670 (+/-0.052) for {'C': 10, 'kernel': 'linear'}
0.670 (+/-0.052) for {'C': 100, 'kernel': 'linear'}
0.670 (+/-0.052) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:


              precision    recall  f1-score   support

           0     0.4444    0.0889

In [310]:
{90:45.36 , 80:48.67, 70:46.69, 60:56.03, 50:52.33}

{90: 45.36, 80: 48.67, 70: 46.69, 60: 56.03, 50: 52.33}

In [ ]:
{90: , 80:, 70:, 60:, 50:}

# Test on other Target
False

In [ ]:
X_test_, y_test_ = get_test_data_and_labels(test_data_file_m,TARGETS_m[2])

In [ ]:
y_true_, y_pred_ = y_test_, clf.predict(X_test_)
print(classification_report(y_true_, y_pred_, digits = 4, labels = [0,1]))